In [1]:
!python -m pip install --upgrade pip
!pip install torch torchvision torchaudio
!pip install -U "nvidia_modelopt[hf]"
!pip install onnx
!pip install onnxruntime-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 46.6 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB 36.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 148.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 124.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 168.1 MB/s  0:00:00
  DEPRECATION: Building 'deepspeed' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` f

In [2]:
!pip install transformers

In [3]:
import time
import torch
from transformers import AutoTokenizer, AutoModel

# 1. Load model & tokenizer
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).eval().cuda()

# 2. Example batch of sentences
text = """ A good story encourages us to turn the next page and read more. We want to find out what happens next and what the main characters do and what they say to each other.
We may feel excited, sad, afraid, angry or really happy. This is because the experience of reading or listening to a story is much more likely to make us 'feel' that we are part
of the story, too. Just like in our 'real' lives, we might love or hate different characters in the story. Perhaps we recognise ourselves or others in some of them. Perhaps we
have similar problems. Because of this natural empathy with the characters, our brains process the reading of stories differently from the way we read factual information.
Our brains don't always recognise the difference between an imagined situation and a real one so the characters become 'alive' to us. What they say and do is therefore more meaningful.
This is why the words and structures that relate a story's events, descriptions and conversations are processed in this deeper way. In fact, cultures all around the world have always
used storytelling to pass knowledge from one generation to another. Our ancestors understood very well that this was the best way to make sure our histories and information about
how to relate to others and to our world was not only understood, but remembered too. (Notice that the word ‘history’ contains the word ‘story’ – More accurately, the word ‘story’
derives from ‘history’.) Encouraging your child to read or listen to stories should therefore help them to learn a second language in a way that is not only fun, but memorable.
Let's take a quick look at learning vocabulary within a factual text or within a story. Imagine the readers are eight-year-olds interested in animals. In your opinion, are they more
likely to remember AND want to continue reading the first or second text? """

texts = [item.strip() for item in text.split(".")][:16]  # adjust batch size here
inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=32).to("cuda")

input_ids = inputs["input_ids"].to(torch.int32)
attention_mask = inputs["attention_mask"].to(torch.int32)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# 3. Baseline PyTorch inference
with torch.no_grad():
    start = time.time()
    baseline_outputs = model(input_ids, attention_mask)
    torch.cuda.synchronize()
    end = time.time()
    baseline_time = end - start
    print(f"PyTorch latency: {baseline_time:.4f} sec")

print("Output shape (PyTorch):", baseline_outputs.last_hidden_state.shape)

PyTorch latency: 0.0421 sec
Output shape (PyTorch): torch.Size([16, 32, 768])


In [ ]:
# save to onnx
output_onnx_file = "roberta.onnx"
torch.onnx.export(
    model.float(),
    (input_ids, attention_mask),
    output_onnx_file,
    input_names=["input_ids", "attention_mask"],
    output_names=["logits", "other"],
    dynamic_axes={
        "input_ids": {0: "batch_size", 1: "seq_len"},
        "attention_mask": {0: "batch_size", 1: "seq_len"},
        "logits": {0: "batch_size", 1: "seq_len"},
        "other": {0: "batch_size"}
    },
    opset_version=17
)

print("save to onnx file:", output_onnx_file)

/tmp/ipython-input-2284662718.py:3: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(


save to onnx file: roberta.onnx


In [ ]:
import onnxruntime as ort
import numpy as np

def run_onnx(onnx_model_pth, provider: str):
    input_ids_np = inputs["input_ids"].to(torch.int32).cpu().numpy()
    attention_mask_np = inputs["attention_mask"].to(torch.int32).cpu().numpy()

    # 4. Onnx inference
    sess = ort.InferenceSession(onnx_model_pth, providers=[provider])

    # Check available providers
    print("Available providers:", ort.get_available_providers())
    print("Current provider:", sess.get_providers())

    start = time.time()
    onnx_outputs = sess.run(None, {"input_ids": input_ids_np, "attention_mask": attention_mask_np})
    end = time.time()
    onnx_time = end - start
    print(f"onnx runtime latency: {onnx_time:.4f} sec")
    return onnx_outputs

def measure_numeric_diff(onnx_tensor, pytorch_base_tensor):
    diff = np.abs(pytorch_base_tensor - onnx_tensor)
    diff_mean = diff.mean()
    diff_max = diff.max()
    diff_percent = (diff > 0.1).mean() * 100

    print("Mean absolute difference:", diff_mean)
    print("Max absolute difference:", diff_max)
    print("Percentage of values with diff > 0.01:", diff_percent, "%")

In [ ]:
onnx_outputs = run_onnx("roberta.onnx", "CUDAExecutionProvider")

onnx_tensor = onnx_outputs[0]  # for last_hidden_state
pytorch_base_tensor = baseline_outputs.last_hidden_state.cpu().numpy()
measure_numeric_diff(onnx_tensor, pytorch_base_tensor)

Available providers: ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
Current provider: ['CUDAExecutionProvider', 'CPUExecutionProvider']
onnx runtime latency: 0.0402 sec
Mean absolute difference: 2.2367885e-07
Max absolute difference: 1.04904175e-05
Percentage of values with diff > 0.01: 0.0 %


In [21]:
onnx_outputs_cpu = run_onnx("roberta.onnx", "CPUExecutionProvider")

onnx_tensor_cpu = onnx_outputs_cpu[0]  # for last_hidden_state
pytorch_base_tensor = baseline_outputs.last_hidden_state.cpu().numpy()
measure_numeric_diff(onnx_tensor_cpu, pytorch_base_tensor)

Available providers: ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
Current provider: ['CPUExecutionProvider']
onnx runtime latency: 1.2183 sec
Mean absolute difference: 3.3164704e-07
Max absolute difference: 3.194809e-05
Percentage of values with diff > 0.01: 0.0 %


In [ ]:
!apt-get install -y build-essential cmake
!pip install --upgrade pip setuptools wheel
!pip install onnxsim --use-pep517 --no-build-isolation



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
  Using cached onnxsim-0.4.36.tar.gz (21.0 MB)
  Preparing metadata (pyproject.toml) ... done
  Created wheel for onnxsim: filename=onnxsim-0.4.36-cp312-cp312-linux_x86_64.whl size=2200462 sha256=6bcdc274eed704238e43a6ae9bfdb172c0fd43f2ebe765850e89e424e3c2e973
  Stored in directory: /root/.cache/pip/wheels/73/5d/cc/db1350d9fabfe7f8442b5d97aff2ff543fc253277f71a6508f
Successfully built onnxsim


In [18]:
!onnxsim roberta.onnx roberta_simplified.onnx

Your model contains "Tile" ops or/and "ConstantOfShape" ops. Folding these ops 
can make the simplified model much larger. If it is not expected, please specify
"--no-large-tensor" (which will lose some optimization chances)
Simplifying...
Finish! Here is the difference:
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃                    ┃ Original Model ┃ Simplified Model ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Add                │ 123            │ 123              │
│ Cast               │ 30             │ 17               │
│ Concat             │ 50             │ 26               │
│ Constant           │ 502            │ 220              │
│ ConstantOfShape    │ 2              │ 0                │
│ CumSum             │ 1              │ 1                │
│ Div                │ 24             │ 24               │
│ Equal              │ 3              │ 3                │
│ Erf                │ 12             │ 12               │
│ Expand            

In [20]:
onnx_outputs_cpu = run_onnx("roberta_simplified.onnx", "CUDAExecutionProvider")

Available providers: ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
Current provider: ['CUDAExecutionProvider', 'CPUExecutionProvider']
onnx runtime latency: 0.0399 sec


In [1]:
import pandas as pd
import numpy as np

data = pd.DataFrame({"Model": ['Pytorch_base', "ONNX", "ONNX", "ONNX_simplified"],
                     "Running_Provider": [np.nan, "CUDAExecutionProvider", "CPUExecutionProvider", "CUDAExecutionProvider"],
                     "Inference_time": [0.0426, 0.0403, 1.2189, 0.0400],
                     "Percentage of diff > 0.01": [np.nan, 0, 0, 0]})
data

,Model,Running_Provider,Inference_time,Percentage of diff > 0.01
0,Pytorch_base,NaN,0.0426,NaN
1,ONNX,CUDAExecutionProvider,0.0403,0.0
2,ONNX,CPUExecutionProvider,1.2189,0.0
3,ONNX_simplified,CUDAExecutionProvider,0.0400,0.0
